In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install git+https://github.com/ourownstory/neural_prophet.git # may take a while
    #!pip install neuralprophet # much faster, but may not have the latest upgrades/bugfixes
    data_location = "https://raw.githubusercontent.com/ourownstory/neural_prophet/master/"
else:
    data_location = "../"

In [2]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
set_log_level("ERROR")

In [3]:
df = pd.read_csv(data_location + "example_data/air_passengers.csv")

In [4]:
folds = NeuralProphet().crossvalidation_split_df(df, freq="MS", k=5, fold_pct=0.1, fold_overlap_pct=0.0)
train, val = [], []
for df_train, df_val in folds:
    m = NeuralProphet(
        seasonality_mode="multiplicative", 
    )
    metrics_train = m.fit(df_train, freq="MS")
    metrics_val = m.test(df_val)
    train.append(metrics_train["MAE"].values[-1])
    val.append(metrics_val["MAE"].values[-1])
#     future = m.make_future_dataframe(df_train, periods=len(df_val), n_historic_predictions=len(df_train))
#     forecast = m.predict(future)
#     fig = m.plot(forecast)

print(train)
print(val)

print(sum(train)/len(train))
print(sum(val)/len(val))

[5.944743259533031, 6.141288930719549, 5.578965056176279, 7.288911950999293, 6.63374401972844]
[27.727863311767578, 5.569243431091309, 24.70100212097168, 11.478766441345215, 29.879167556762695]
6.3175306434313185
19.871208572387694
